In [151]:
import pandas as pd
import numpy as np

In [92]:
combined = pd.read_csv(r'../Fantasy-Premier-League/data/2023-24/gws/merged_gw.csv')

combined.columns.values

array(['name', 'position', 'team', 'xP', 'assists', 'bonus', 'bps',
       'clean_sheets', 'creativity', 'element', 'expected_assists',
       'expected_goal_involvements', 'expected_goals',
       'expected_goals_conceded', 'fixture', 'goals_conceded',
       'goals_scored', 'ict_index', 'influence', 'kickoff_time',
       'minutes', 'opponent_team', 'own_goals', 'penalties_missed',
       'penalties_saved', 'red_cards', 'round', 'saves', 'selected',
       'starts', 'team_a_score', 'team_h_score', 'threat', 'total_points',
       'transfers_balance', 'transfers_in', 'transfers_out', 'value',
       'was_home', 'yellow_cards', 'GW'], dtype=object)

In [120]:
combined.head(2)

,name,position,team,xP,assists,bonus,bps,clean_sheets,creativity,element,...,team_h_score,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,yellow_cards,GW
0,Femi Seriki,DEF,Sheffield Utd,0.5,0,0,0,0,0.0,653,...,0,0.0,0,0,0,0,40,True,0,1
1,Jack Hinshelwood,MID,Brighton,1.5,0,0,0,0,0.0,621,...,4,0.0,0,0,0,0,45,True,0,1


In [121]:
combined = combined[combined['GW'] != 4]
validation = combined[combined['GW'] == 4]

In [124]:
# todo: dron na 
y = combined[['total_points']]
X = combined[['position', 'value', 'was_home']]  # todo: add opponent

X_val = validation[['position', 'value', 'was_home']]
y_val = validation[['total_points']]

In [126]:
X.loc[X['was_home'] == True, 'was_home'] = 1
X.loc[X['was_home'] == False, 'was_home'] = 0

X_val.loc[X_val['was_home'] == True, 'was_home'] = 1
X_val.loc[X_val['was_home'] == False, 'was_home'] = 0

In [127]:
X.columns = X.columns.astype(str)
X_val.columns = X_val.columns.astype(str)

In [130]:
combined[combined['GW'] == 4]

,name,position,team,xP,assists,bonus,bps,clean_sheets,creativity,element,...,team_h_score,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,yellow_cards,GW


In [131]:
# todo: encoding categoricals
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer

transformer = make_column_transformer(
    (OneHotEncoder(), ['position']), remainder='passthrough'
)

transformed = transformer.fit_transform(X)
transformed_df = pd.DataFrame(
    transformed,
    columns=transformer.get_feature_names_out()
)

# transformed_val = transformer.fit_transform(X_val)
# transformed_val_df = pd.DataFrame(
#     transformed_val,
#     columns=transformer.get_feature_names_out()
# )

In [133]:
transformed_df.columns

Index(['onehotencoder__position_DEF', 'onehotencoder__position_FWD',
       'onehotencoder__position_GK', 'onehotencoder__position_MID',
       'remainder__value', 'remainder__was_home'],
      dtype='object')

In [134]:
transformed_df.head(2)

,onehotencoder__position_DEF,onehotencoder__position_FWD,onehotencoder__position_GK,onehotencoder__position_MID,remainder__value,remainder__was_home
0,1.0,0.0,0.0,0.0,40,1
1,0.0,0.0,0.0,1.0,45,1


In [135]:
X = transformed_df

In [142]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=.3,
    shuffle=True,
    random_state=42
)

In [143]:
from sklearn.ensemble import RandomForestRegressor

reg = RandomForestRegressor(n_estimators=100, verbose=0, criterion='squared_error')
reg.fit(X_train, y_train)

/home/rex/Desktop/test/FPL/venv/lib/python3.9/site-packages/sklearn/base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForestRegressor()

In [144]:
pred = reg.predict(X_test)


In [145]:
reg.score(X_test, y_test)

0.07974185529276268

In [146]:
from sklearn.metrics import mean_absolute_error

mean_absolute_error(y_test, pred)

1.2910230659890984

In [154]:
X[:1].shape


(1, 6)

In [155]:


reg.predict(
    np.array([0, 0, 0, 1, 125, 1]).reshape(1, -1)
)

/home/rex/Desktop/test/FPL/venv/lib/python3.9/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


array([4.26])